In [1]:
import os
import sys
import requests

sys.path.append("../")

from settings import USER_AGENT, ITEMS_ENDPOINT, EXCHANGE_ENDPOINT, FETCH_ENDPOINT
from helpers import Queries, ResultParser

from utils.queries import QueryBuilder
from utils.parsers import Parser

In [2]:
class Client:
    def __init__(self):
        self.headers = {
            "Content-Type": "application/json",
            "User-Agent": USER_AGENT
        }
        self.queries = QueryBuilder()
        self.parser = Parser()
    
    def query(self, **kwargs):
        raise NotImplementedError()

In [3]:
class ExchangeClient(Client):    
    def __init__(self):
       super().__init__()
       self.search_endpoint = EXCHANGE_ENDPOINT

    def query(self, **kwargs):
        base_currency = kwargs.pop("base_currency")
        quote_currency = kwargs.pop("quote_currency")

        bq_result = self.__search(buy=base_currency, sell=quote_currency)
        parsed_bq_result = self.parser.parse(bq_result, how="exchange")

        qb_result = self.__search(buy=quote_currency, sell=base_currency)
        parsed_qb_result = self.parser.parse(qb_result, how="exchange")
        
        return parsed_bq_result, parsed_qb_result

    def __search(self, buy, sell):
        try:
            resp = requests.post(
                url = self.search_endpoint,
                headers=self.headers,
                json = self.queries.get_exchange(buy, sell)
            )
            resp.raise_for_status()
        except requests.exceptions.HTTPError as e:
            raise SystemExit(e)

        results = list(resp.json()["result"].values())
        return results

In [4]:
cl = ExchangeClient()

bq, qb = cl.query(base_currency="chaos", quote_currency="exalted")

bq

[{'id': '89f8d01b441156ff96159e79ed7fa69a22727654139c276d11a26276cdc7d3fe',
  'indexed': '2024-08-06T18:23:10+00:00',
  'whisper': '@RetirementNextFriday Здравствуйте, хочу купить у вас {0} за {1} в лиге Settlers',
  'pay': {'amount': 10, 'currency': 'exalted'},
  'get': {'amount': 131, 'currency': 'chaos', 'stock': 432},
  'exch_rate': 0.07633587786259542},
 {'id': 'be29587dc3c09ce62bc4c6f2e7dce7ad755c20ed11a9de9f16bf4f97baf248db',
  'indexed': '2024-08-06T18:17:43+00:00',
  'whisper': "@Lukopix Hi, I'd like to buy your {0} for my {1} in Settlers",
  'pay': {'amount': 1, 'currency': 'exalted'},
  'get': {'amount': 13, 'currency': 'chaos', 'stock': 15},
  'exch_rate': 0.07692307692307693},
 {'id': '2aed80f9aa8111fd0ab14384355a1e30d8683066cf63a92f073078a4feb6580b',
  'indexed': '2024-08-06T17:17:05+00:00',
  'whisper': "@INSANECAT_BOATYEP Hi, I'd like to buy your {0} for my {1} in Settlers",
  'pay': {'amount': 5, 'currency': 'exalted'},
  'get': {'amount': 65, 'currency': 'chaos', 'sto

In [5]:
class ItemsClient(Client):    
    def __init__(self):
       super().__init__()
       self.search_endpoint = ITEMS_ENDPOINT
       self.fetch_endpoint = FETCH_ENDPOINT

    def query(self, **kwargs):
        search_results = self.__search(**kwargs)
        fetch_results = self.__fetch(search_results, n_limit=10)
        parsed_results = self.parser.parse(fetch_results, "simple")

        return parsed_results

    def __search(self, **kwargs):
        json_data = self.queries.get_items(**kwargs)

        try:
            resp = requests.post(
                url=self.search_endpoint,
                headers=self.headers,
                json=json_data
            )
            resp.raise_for_status()
        except requests.exceptions.HTTPError as e:
            raise SystemExit(e)
        
        search_json = resp.json()
        search_results = search_json["result"]
        return search_results

    def __fetch(self, search_results, n_limit=10):
        search_results = search_results[:n_limit]
        results_str = ",".join(search_results)

        try:
            resp = requests.get(
                url=os.path.join(self.fetch_endpoint, results_str),
                headers=self.headers,
            )
            resp.raise_for_status()
        except requests.exceptions.HTTPError as e:
            raise SystemExit(e)
    
        fetch_json = resp.json()
        fetch_results = fetch_json["result"]

        return fetch_results

In [6]:
cl = ItemsClient()
results = cl.query(
    item_type="Infernal Sword"
)

results

[{'id': '13acd91af378c14c21538e8c0c32f35c565c6aee29500fbb247f2f7802787e72',
  'indexed': '2024-08-06T16:31:19Z',
  'price': {'amount': 1, 'currency': 'alch'},
  'item': {'name': '',
   'type': 'Infernal Sword',
   'rarity': 'Unique',
   'ilvl': 82}},
 {'id': 'bf47d3bec6258fcfdec8571eb3b7d3cfd56bd6b320affbdd00f6b2039fa1281f',
  'indexed': '2024-08-05T04:41:40Z',
  'price': {'amount': 1, 'currency': 'alch'},
  'item': {'name': "Oro's Sacrifice",
   'type': 'Infernal Sword',
   'rarity': 'Unique',
   'ilvl': 75}},
 {'id': '24af675bf6b998460f837cf6699e64dbdbcfb914bfe8185d0f94df8c7599179d',
  'indexed': '2024-08-04T11:09:24Z',
  'price': {'amount': 1, 'currency': 'alch'},
  'item': {'name': "Oro's Sacrifice",
   'type': 'Infernal Sword',
   'rarity': 'Unique',
   'ilvl': 72}},
 {'id': 'dfd81ab7f12e7b049a62f8d0de47dc6c5ebfe2511faa66eba76623501da33b5f',
  'indexed': '2024-08-06T17:48:47Z',
  'price': {'amount': 1, 'currency': 'alch'},
  'item': {'name': "Oro's Sacrifice",
   'type': 'Infernal